In [42]:
# Cell 1: Import & Configuration
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

# --- Global Settings (ตั้งค่าตรงนี้ทีเดียว) ---
YEARS = 3              # ต้องการทำ Projection กี่ปี (เปลี่ยนเลขตรงนี้ได้เลย)
R_EXPECTED = 0.10      # ผลตอบแทนคาดหวัง (Discount Rate) 10%
GROWTH_RATE = 0.04     # สมมติการเติบโตของปันผลและราคา 4% ต่อปี

# รายชื่อหุ้น SET50
set50_tickers = [
    "ADVANC", "AOT", "AWC", "BANPU", "BBL", "BDMS", "BEM", "BGRIM", "BH", "BJC",
    "BTS", "CBG", "CENTEL", "COM7", "CPALL", "CPF", "CPN", "CRC", "DELTA", "EA",
    "EGCO", "GLOBAL", "GPSC", "GULF", "HMPRO", "INTUCH", "IVL", "KBANK", "KCE", "KTB",
    "KTC", "LH", "MINT", "MTC", "OR", "OSP", "PTT", "PTTEP", "PTTGC", "RATCH",
    "SAWAD", "SCB", "SCC", "SCGP", "TISCO", "TOP", "TRUE", "TTB", "TU", "WHA"
]

print(f"✅ Configuration Loaded: Forecasting {YEARS} years with {R_EXPECTED*100}% Discount Rate.")

✅ Configuration Loaded: Forecasting 3 years with 10.0% Discount Rate.


In [43]:
# Cell 2: Define Valuation Function
def calculate_multistage_ddm(symbol, discount_rate, growth_rate, years):
    """
    ฟังก์ชันคำนวณ DDM แบบ Dynamic ตามจำนวนปีที่กำหนด
    """
    try:
        ticker_symbol = f"{symbol}.BK"
        stock = yf.Ticker(ticker_symbol)
        
        # 1. Get Current Price
        current_price = stock.fast_info['last_price']
        if current_price is None: return None
        
        # 2. Get Trailing Dividend (D0)
        dividends = stock.dividends
        one_year_ago = pd.Timestamp.now(tz=datetime.timezone.utc) - pd.DateOffset(years=1)
        d0 = dividends[dividends.index >= one_year_ago].sum()
        
        # ถ้าไม่มีปันผล ให้ใส่ 0 (ระวัง: โมเดลนี้จะไม่ work กับหุ้นที่ไม่จ่ายปันผล)
        if d0 == 0: d0 = 0

        # --- Dynamic Calculation Loop ---
        total_pv_dividends = 0
        current_d = d0
        
        # Loop คำนวณ V1, V2, ..., Vn
        for year in range(1, years + 1):
            # Grow Dividend
            current_d = current_d * (1 + growth_rate)
            
            # Discount back to present value
            pv_d = current_d / ((1 + discount_rate) ** year)
            
            # Summation
            total_pv_dividends += pv_d

        # 3. Calculate Terminal Value (Pn)
        # สมมติราคาปีสุดท้าย = ราคาปัจจุบัน * (1+g)^n
        pn_future = current_price * ((1 + growth_rate) ** years)
        
        # Discount Terminal Value back to present
        pv_terminal = pn_future / ((1 + discount_rate) ** years)
        
        # 4. Total Intrinsic Value
        intrinsic_value = total_pv_dividends + pv_terminal
        
        return {
            "Symbol": symbol,
            "Current Price": current_price,
            "D0": d0,
            "Sum_PV_Dividends": total_pv_dividends, # มูลค่ารวมของปันผลตลอด n ปี
            "PV_Terminal_Price": pv_terminal,       # มูลค่าปัจจุบันของราคาขายปีสุดท้าย
            "Intrinsic Value": intrinsic_value,
            "Future Price (Pn)": pn_future
        }

    except Exception as e:
        # print(f"Error fetching {symbol}: {e}") # Uncomment to debug
        return None

print("✅ Valuation Function Defined.")

✅ Valuation Function Defined.


In [44]:
# Cell 3: Execute Valuation for SET50
results = []
print(f"🚀 Processing {len(set50_tickers)} stocks... (This may take 1-2 minutes)")

for stock in set50_tickers:
    res = calculate_multistage_ddm(
        symbol=stock, 
        discount_rate=R_EXPECTED, 
        growth_rate=GROWTH_RATE, 
        years=YEARS
    )
    if res:
        results.append(res)
    else:
        print(f"⚠️ Could not fetch data for {stock}")

# Create DataFrame
df_results = pd.DataFrame(results)

# Calculate Upside/Downside
if not df_results.empty:
    df_results['Upside (%)'] = (df_results['Intrinsic Value'] - df_results['Current Price']) / df_results['Current Price'] * 100
    df_results['Recommendation'] = np.where(df_results['Upside (%)'] > 0, 'Undervalue', 'Overvalue')
    
    # Sort by Upside
    df_results = df_results.sort_values(by="Upside (%)", ascending=False)
    
print("✅ Calculation Complete.")

🚀 Processing 50 stocks... (This may take 1-2 minutes)


$INTUCH.BK: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$INTUCH.BK: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


⚠️ Could not fetch data for INTUCH
✅ Calculation Complete.


In [45]:
# Cell 4: Display Results with Styling
def make_clickable(val):
    # (Optional) ถ้าอยากให้ลิสต์ดูสวยขึ้น
    return val

# เลือก Column ที่จะแสดง
cols = ["Symbol", "Current Price", "Intrinsic Value", "Upside (%)", "Recommendation", "Sum_PV_Dividends", "PV_Terminal_Price"]

if not df_results.empty:
    # สร้าง Style
    styled_df = df_results[cols].style.format({
        "Current Price": "{:.2f}",
        "Intrinsic Value": "{:.2f}",
        "Upside (%)": "{:+.2f}%",
        "Sum_PV_Dividends": "{:.2f}",
        "PV_Terminal_Price": "{:.2f}"
    }).background_gradient(subset=["Upside (%)"], cmap="RdYlGn", vmin=-20, vmax=20) \
      .applymap(lambda v: 'color: green; font-weight: bold' if v == 'Undervalue' else 'color: red', subset=['Recommendation'])

    print(f"📊 Valuation Results ({YEARS}-Year Horizon):")
    display(styled_df) # ถ้าใช้ใน VS Code หรือ Colab ใช้คำสั่ง display() ได้เลย
else:
    print("❌ No data found.")

📊 Valuation Results (3-Year Horizon):


C:\Users\natta\AppData\Local\Temp\ipykernel_9476\2575786760.py:18: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(lambda v: 'color: green; font-weight: bold' if v == 'Undervalue' else 'color: red', subset=['Recommendation'])


,Symbol,Current Price,Intrinsic Value,Upside (%),Recommendation,Sum_PV_Dividends,PV_Terminal_Price
36,PTTEP,116.50,123.22,+5.77%,Undervalue,24.76,98.46
30,LH,3.86,4.07,+5.38%,Undervalue,0.81,3.26
40,SCB,139.00,145.50,+4.68%,Undervalue,28.03,117.47
15,CPF,21.60,22.42,+3.78%,Undervalue,4.16,18.25
17,CRC,18.50,19.12,+3.38%,Undervalue,3.49,15.63
43,TISCO,111.00,114.61,+3.26%,Undervalue,20.80,93.81
28,KTB,29.00,29.81,+2.79%,Undervalue,5.30,24.51
35,PTT,33.25,34.01,+2.27%,Undervalue,5.91,28.10
26,KBANK,191.50,195.40,+2.04%,Undervalue,33.56,161.84
27,KCE,18.60,18.94,+1.83%,Undervalue,3.22,15.72


In [47]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

# --- Config ---
YEARS = 3
R_EXPECTED = 0.10      # ผลตอบแทนคาดหวัง (k)
GROWTH_RATE = 0.04     # อัตราการเติบโต (g)
# หมายเหตุ: ตามทฤษฎี Gordon Growth -> r ต้องมากกว่า g เสมอ (0.10 > 0.04 ผ่าน ✅)

set50_tickers = [
    "ADVANC", "AOT", "AWC", "BANPU", "BBL", "BDMS", "BEM", "BGRIM", "BH", "BJC",
    "BTS", "CBG", "CENTEL", "COM7", "CPALL", "CPF", "CPN", "CRC", "DELTA", "EA",
    "EGCO", "GLOBAL", "GPSC", "GULF", "HMPRO", "INTUCH", "IVL", "KBANK", "KCE", "KTB",
    "KTC", "LH", "MINT", "MTC", "OR", "OSP", "PTT", "PTTEP", "PTTGC", "RATCH",
    "SAWAD", "SCB", "SCC", "SCGP", "TISCO", "TOP", "TRUE", "TTB", "TU", "WHA"
]

results = []
print(f"🚀 Calculating Pure DDM (Independent of Market Price)...")

for symbol in set50_tickers:
    try:
        stock = yf.Ticker(f"{symbol}.BK")
        current_price = stock.fast_info['last_price']
        
        # หา D0
        dividends = stock.dividends
        one_year_ago = pd.Timestamp.now(tz=datetime.timezone.utc) - pd.DateOffset(years=1)
        d0 = dividends[dividends.index >= one_year_ago].sum()
        if d0 == 0: d0 = 0

        # --- ส่วนที่ 1: คิดลดปันผลระหว่างทาง (Dividends PV) ---
        total_pv_dividends = 0
        current_d = d0
        d_list = []
        
        for i in range(1, YEARS + 1):
            current_d = current_d * (1 + GROWTH_RATE) # โตไปเรื่อยๆ
            d_list.append(current_d)
            pv = current_d / ((1 + R_EXPECTED) ** i)
            total_pv_dividends += pv
            
        # --- ส่วนที่ 2: คิด Terminal Value แบบ Pure DDM (Gordon Growth) ---
        # หา D ปีถัดไป (ปีที่ n+1)
        d_next = current_d * (1 + GROWTH_RATE)
        
        # สูตร Gordon Growth หา Pn (ราคา ณ ปีที่ n) โดยไม่ง้อราคาตลาด
        pn_theoretical = d_next / (R_EXPECTED - GROWTH_RATE)
        
        # คิดลด Pn กลับมาปัจจุบัน
        pv_terminal = pn_theoretical / ((1 + R_EXPECTED) ** YEARS)
        
        # --- รวมมูลค่า ---
        intrinsic_value = total_pv_dividends + pv_terminal
        
        # คำนวณ Upside
        upside = (intrinsic_value - current_price) / current_price
        
        results.append({
            "Symbol": symbol,
            "Current Price": current_price,
            "D1": d_list[0], "D2": d_list[1], "D3": d_list[2],
            "Terminal Value (Pn)": pn_theoretical, # ราคาขายปีสุดท้ายที่คำนวณจากปันผลล้วนๆ
            "Intrinsic Value": intrinsic_value,
            "Upside": upside,
            "Meaning": 'Undervalue' if upside > 0 else 'Overvalue'
        })

    except Exception as e:
        print(f"Error {symbol}: {e}")

df = pd.DataFrame(results).sort_values(by="Upside", ascending=False)

print("\n🌍 Pure DDM Valuation (World Safe Edition):")
display(df.style.format({
    "Current Price": "{:.2f}",
    "D1": "{:.2f}", "D2": "{:.2f}", "D3": "{:.2f}",
    "Terminal Value (Pn)": "{:.2f}",
    "Intrinsic Value": "{:.2f}",
    "Upside": "{:+.2%}"
}).background_gradient(subset=["Upside"], cmap="RdYlGn"))

🚀 Calculating Pure DDM (Independent of Market Price)...


$INTUCH.BK: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$INTUCH.BK: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


Error INTUCH: 'currentTradingPeriod'

🌍 Pure DDM Valuation (World Safe Edition):


,Symbol,Current Price,D1,D2,D3,Terminal Value (Pn),Intrinsic Value,Upside,Meaning
36,PTTEP,116.50,9.59,9.98,10.38,179.87,159.90,+37.25%,Undervalue
30,LH,3.94,0.31,0.32,0.34,5.85,5.20,+31.98%,Undervalue
40,SCB,140.50,10.86,11.29,11.74,203.56,180.96,+28.80%,Undervalue
15,CPF,21.20,1.61,1.68,1.74,30.22,26.87,+26.73%,Undervalue
43,TISCO,111.00,8.06,8.38,8.72,151.11,134.33,+21.02%,Undervalue
28,KTB,29.00,2.05,2.14,2.22,38.51,34.23,+18.05%,Undervalue
17,CRC,19.40,1.35,1.41,1.46,25.35,22.53,+16.15%,Undervalue
35,PTT,33.50,2.29,2.38,2.47,42.89,38.13,+13.83%,Undervalue
26,KBANK,192.50,13.00,13.52,14.06,243.72,216.67,+12.55%,Undervalue
27,KCE,18.50,1.25,1.30,1.35,23.40,20.80,+12.43%,Undervalue


In [48]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

# ==========================================
# ⚙️ ส่วนตั้งค่า (Config) - ปรับแต่งได้ที่นี่
# ==========================================
YEARS = 3              # ล็อคไว้ 3 ปี (เพื่อให้ได้ D1, D2, D3)
R_EXPECTED = 0.10      # ผลตอบแทนคาดหวัง (10%)
GROWTH_RATE = 0.04     # อัตราการเติบโต (4%)

# รายชื่อหุ้น SET50 (อัปเดตล่าสุด)
set50_tickers = [
    "ADVANC", "AOT", "AWC", "BANPU", "BBL", "BDMS", "BEM", "BGRIM", "BH", "BJC",
    "BTS", "CBG", "CENTEL", "COM7", "CPALL", "CPF", "CPN", "CRC", "DELTA", "EA",
    "EGCO", "GLOBAL", "GPSC", "GULF", "HMPRO", "INTUCH", "IVL", "KBANK", "KCE", "KTB",
    "KTC", "LH", "MINT", "MTC", "OR", "OSP", "PTT", "PTTEP", "PTTGC", "RATCH",
    "SAWAD", "SCB", "SCC", "SCGP", "TISCO", "TOP", "TRUE", "TTB", "TU", "WHA"
]

# ==========================================
# 🧠 ส่วนคำนวณ (Calculation Core)
# ==========================================
def calculate_ddm_market_based(symbol):
    try:
        # 1. ดึงข้อมูลราคาและปันผล
        stock = yf.Ticker(f"{symbol}.BK")
        current_price = stock.fast_info['last_price']
        
        if current_price is None: return None
        
        # หา D0 (ปันผล 1 ปีย้อนหลัง)
        dividends = stock.dividends
        one_year_ago = pd.Timestamp.now(tz=datetime.timezone.utc) - pd.DateOffset(years=1)
        d0 = dividends[dividends.index >= one_year_ago].sum()
        if d0 == 0: d0 = 0 # กันเหนียวกรณีไม่เจอข้อมูล

        # 2. ประมาณการปันผลอนาคต (D1, D2, D3)
        d1 = d0 * (1 + GROWTH_RATE)
        d2 = d1 * (1 + GROWTH_RATE)
        d3 = d2 * (1 + GROWTH_RATE)
        
        # 3. คิดลดมูลค่าปันผล (PV of Dividends)
        v1 = d1 / ((1 + R_EXPECTED) ** 1)
        v2 = d2 / ((1 + R_EXPECTED) ** 2)
        v3 = d3 / ((1 + R_EXPECTED) ** 3)
        
        # 4. คิด Terminal Value (Pn) แบบ Market Based
        # สมมติราคาหุ้นโตตาม Growth Rate (Practical Model)
        pn_future = current_price * ((1 + GROWTH_RATE) ** YEARS)
        pn_discounted = pn_future / ((1 + R_EXPECTED) ** YEARS)
        
        # 5. รวมมูลค่าที่แท้จริง (Pred)
        pred = v1 + v2 + v3 + pn_discounted
        
        # 6. คำนวณ % Diff (สูตรตาม Excel ของคุณ: (Pred - Price) / Pred)
        diff_percent = (pred - current_price) / pred
        meaning = 'Undervalue' if diff_percent > 0 else 'Overvalue'

        return {
            "Symbol": symbol,
            "Current Price": current_price,
            "D1": d1,
            "D2": d2,
            "D3": d3,
            "Pred": pred,
            "diff_percent": diff_percent,
            "meaning": meaning
        }

    except Exception as e:
        return None

# ==========================================
# 🚀 ส่วนแสดงผล (Execution & Display)
# ==========================================
results = []
print(f"กำลังประมวลผลหุ้น SET50 ทั้งหมด... (รอสักครู่)")

for stock in set50_tickers:
    res = calculate_ddm_market_based(stock)
    if res:
        results.append(res)

df_final = pd.DataFrame(results)

# เรียงลำดับเอาตัว Undervalue มากสุดขึ้นก่อน
df_final = df_final.sort_values(by="diff_percent", ascending=False)

# จัดรูปแบบการแสดงผล (Styling)
cols_ordered = ["Symbol", "Current Price", "D1", "D2", "D3", "Pred", "diff_percent", "meaning"]

print(f"\n✅ เสร็จสิ้น! นี่คือผลลัพธ์แบบ Market-Based DDM:")
display(
    df_final[cols_ordered].style.format({
        "Current Price": "{:.2f}",
        "D1": "{:.2f}",
        "D2": "{:.2f}",
        "D3": "{:.2f}",
        "Pred": "{:.2f}",
        "diff_percent": "{:.2%}"
    }).background_gradient(subset=["diff_percent"], cmap="RdYlGn", vmin=-0.2, vmax=0.2) \
      .applymap(lambda v: 'color: green; font-weight: bold' if v == 'Undervalue' else 'color: red', subset=['meaning'])
)

กำลังประมวลผลหุ้น SET50 ทั้งหมด... (รอสักครู่)


$INTUCH.BK: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$INTUCH.BK: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")



✅ เสร็จสิ้น! นี่คือผลลัพธ์แบบ Market-Based DDM:


C:\Users\natta\AppData\Local\Temp\ipykernel_9476\1731702747.py:104: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(lambda v: 'color: green; font-weight: bold' if v == 'Undervalue' else 'color: red', subset=['meaning'])


,Symbol,Current Price,D1,D2,D3,Pred,diff_percent,meaning
36,PTTEP,116.50,9.59,9.98,10.38,123.22,5.45%,Undervalue
30,LH,3.92,0.31,0.32,0.34,4.12,4.81%,Undervalue
40,SCB,140.00,10.86,11.29,11.74,146.34,4.33%,Undervalue
15,CPF,21.10,1.61,1.68,1.74,21.99,4.06%,Undervalue
43,TISCO,110.50,8.06,8.38,8.72,114.19,3.23%,Undervalue
28,KTB,29.00,2.05,2.14,2.22,29.81,2.72%,Undervalue
17,CRC,19.50,1.35,1.41,1.46,19.97,2.35%,Undervalue
35,PTT,33.50,2.29,2.38,2.47,34.22,2.10%,Undervalue
27,KCE,18.40,1.25,1.30,1.35,18.77,1.98%,Undervalue
26,KBANK,192.50,13.00,13.52,14.06,196.24,1.91%,Undervalue


In [50]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

# ==========================================
# ⚙️ ตั้งค่า (Config) - อยากได้กี่ปีแก้ตรงนี้
# ==========================================
YEARS = 3             # <--- ลองเปลี่ยนเลขตรงนี้ดูครับ (เช่น 5, 10)
R_EXPECTED = 0.10      # ผลตอบแทนคาดหวัง (10%)
GROWTH_RATE = 0.04     # อัตราการเติบโต (4%)

# รายชื่อหุ้น (ตัวอย่าง)
set50_tickers = [
    "ADVANC", "AOT", "BBL", "BDMS", "CPALL", "CPF", "CPN", "DELTA", "GULF",
    "KBANK", "KTB", "LH", "MINT", "PTT", "PTTEP", "SCB", "SCC", "TISCO"
]

# ==========================================
# 🧠 ส่วนคำนวณ (Dynamic Calculation Loop)
# ==========================================
def calculate_ddm_dynamic(symbol, years):
    try:
        # 1. ดึงข้อมูล
        stock = yf.Ticker(f"{symbol}.BK")
        current_price = stock.fast_info['last_price']
        
        if current_price is None: return None
        
        # หา D0 (ปันผล 1 ปีย้อนหลัง)
        dividends = stock.dividends
        one_year_ago = pd.Timestamp.now(tz=datetime.timezone.utc) - pd.DateOffset(years=1)
        d0 = dividends[dividends.index >= one_year_ago].sum()
        if d0 == 0: d0 = 0 

        # 2. Loop คำนวณปันผลแต่ละปี (D1...Dn)
        total_pv_dividends = 0
        current_d = d0
        dynamic_columns = {} # เก็บค่า D1, D2... เพื่อไปโชว์ในตาราง
        
        for i in range(1, years + 1):
            # ปันผลเติบโต
            current_d = current_d * (1 + GROWTH_RATE)
            
            # เก็บค่าไว้แสดงในตาราง
            dynamic_columns[f"D{i}"] = current_d
            
            # คิดลดเงินปันผลกลับมาเป็นค่าปัจจุบัน
            pv = current_d / ((1 + R_EXPECTED) ** i)
            total_pv_dividends += pv
        
        # 3. คิด Terminal Value (Pn) ณ ปีสุดท้าย
        # ใช้สูตร Market Based: ราคาอนาคต = ราคาปัจจุบัน * Growth^ปี
        pn_future = current_price * ((1 + GROWTH_RATE) ** years)
        pn_discounted = pn_future / ((1 + R_EXPECTED) ** years)
        
        # 4. รวมมูลค่า (Pred)
        pred = total_pv_dividends + pn_discounted
        
        # 5. คำนวณ % Diff (สูตร Excel: (Pred - Price) / Pred)
        diff_percent = (pred - current_price) / pred
        meaning = 'Undervalue' if diff_percent > 0 else 'Overvalue'

        # สร้าง Dictionary ผลลัพธ์
        result = {
            "Symbol": symbol,
            "Current Price": current_price,
            "Pred": pred,
            "diff_percent": diff_percent,
            "meaning": meaning
        }
        # เอา D1, D2... ยัดเข้าไปในผลลัพธ์
        result.update(dynamic_columns)
        
        return result

    except Exception as e:
        return None

# ==========================================
# 🚀 แสดงผล (Auto Table Format)
# ==========================================
results = []
print(f"กำลังคำนวณแบบ {YEARS} ปี สำหรับหุ้น {len(set50_tickers)} ตัว...")

for stock in set50_tickers:
    res = calculate_ddm_dynamic(stock, YEARS)
    if res:
        results.append(res)

if results:
    df = pd.DataFrame(results)
    
    # เรียงลำดับ Undervalue มากสุดขึ้นก่อน
    df = df.sort_values(by="diff_percent", ascending=False)
    
    # จัดลำดับ Column ให้สวยงาม (เอา D1...Dn มาแทรกตรงกลาง)
    d_cols = [f"D{i}" for i in range(1, YEARS + 1)]
    cols_ordered = ["Symbol", "Current Price"] + d_cols + ["Pred", "diff_percent", "meaning"]
    
    # สร้างตาราง
    print(f"\n✅ ผลลัพธ์การประเมินมูลค่า ({YEARS} ปี):")
    
    # เตรียม Format สำหรับ D1...Dn ให้เป็นทศนิยม 2 ตำแหน่ง
    format_dict = {
        "Current Price": "{:.2f}",
        "Pred": "{:.2f}",
        "diff_percent": "{:.2%}"
    }
    for col in d_cols:
        format_dict[col] = "{:.2f}"
        
    display(
        df[cols_ordered].style.format(format_dict)
        .background_gradient(subset=["diff_percent"], cmap="RdYlGn", vmin=-0.2, vmax=0.2)
        .applymap(lambda v: 'color: green; font-weight: bold' if v == 'Undervalue' else 'color: red', subset=['meaning'])
    )
else:
    print("ไม่พบข้อมูล")

กำลังคำนวณแบบ 3 ปี สำหรับหุ้น 18 ตัว...

✅ ผลลัพธ์การประเมินมูลค่า (3 ปี):


C:\Users\natta\AppData\Local\Temp\ipykernel_9476\2221302030.py:116: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(lambda v: 'color: green; font-weight: bold' if v == 'Undervalue' else 'color: red', subset=['meaning'])


,Symbol,Current Price,D1,D2,D3,Pred,diff_percent,meaning
14,PTTEP,116.50,9.59,9.98,10.38,123.22,5.45%,Undervalue
11,LH,3.92,0.31,0.32,0.34,4.12,4.81%,Undervalue
15,SCB,140.00,10.86,11.29,11.74,146.34,4.33%,Undervalue
5,CPF,21.10,1.61,1.68,1.74,21.99,4.06%,Undervalue
17,TISCO,110.50,8.06,8.38,8.72,114.19,3.23%,Undervalue
10,KTB,29.00,2.05,2.14,2.22,29.81,2.72%,Undervalue
13,PTT,33.50,2.29,2.38,2.47,34.22,2.10%,Undervalue
9,KBANK,192.50,13.00,13.52,14.06,196.24,1.91%,Undervalue
2,BBL,170.50,8.84,9.19,9.56,166.91,-2.15%,Overvalue
3,BDMS,20.20,0.78,0.81,0.84,19.08,-5.84%,Overvalue
